## ⚔️ Zwei Spieler mit Strategien $V$ und $W$: Erwartete Spieldauer $E(V, W)$

In dieser Variante spielen zwei Personen gegeneinander. Jede Person verteilt $n$ Chips auf $m$ Felder:

- Spieler A wählt eine Verteilung $V = (v_1, \dots, v_m)$
- Spieler B wählt eine Verteilung $W = (w_1, \dots, w_m)$

Ein Wurf trifft Feld $j$ mit Wahrscheinlichkeit $p_j$.
Falls auf diesem Feld bei einem der Spieler ein Chip liegt, wird **ein Chip entfernt** – ggf. bei beiden Spielern gleichzeitig.

---

### 🛑 Abbruchbedingungen:

Das Spiel endet, sobald eine dieser Situationen eintritt:

- $V = W$ (z. B. durch Symmetrie oder Restgleichheit)
- $\sum_j v_j = 0$ (Spieler A hat keine Chips mehr)
- $\sum_j w_j = 0$ (Spieler B hat keine Chips mehr)

Dann gilt:

$$
E(V, W) = 0
$$

---

### 🔁 Rekursive Formel:

Für alle anderen Fälle wird rekursiv gerechnet:

$$
E(V, W) = \frac{1}{s} + \frac{1}{s}\sum_{j=1}^m q_j \cdot E(V - \delta_j^V, W - \delta_j^W)
$$

mit:

- $q_j := p_j \cdot \mathbf{1}_{\{v_j > 0 \lor w_j > 0\}}$
- $s := \sum_{j=1}^m q_j$ ist die Gesamtwahrscheinlichkeit, dass ein Wurf überhaupt etwas bewirkt.

- $\delta_j^V = e_j$ falls $v_j > 0$, sonst $0$
- $\delta_j^W = e_j$ falls $w_j > 0$, sonst $0$

---

### 🧠 Bedeutung:

- Der Ausdruck zählt, wie viele Würfe **im Durchschnitt nötig** sind, bis das Spiel (unter den genannten Bedingungen) endet.
- Die Formel berücksichtigt nur Felder, auf denen bei mindestens einem Spieler Chips liegen – sonst wirkt ein Wurf nicht.

Dies bildet den realen Spielverlauf exakt ab und liefert einen **bedingten Erwartungswert** über alle möglichen Wirkungen des nächsten Wurfs.


---
## Programm für E(V,W) ##
---

In [13]:
from fractions import Fraction
from functools import lru_cache

@lru_cache(maxsize=None)
def expected_duel(V, W, p):
    # Abbruchbedingungen: einer leer oder V = W
    if V == W or all(v == 0 for v in V) or all(w == 0 for w in W):
        return Fraction(0)
    
    total = Fraction(0)
    s = Fraction(0)
    
    for j in range(len(p)):
        vj, wj = V[j], W[j]
        if vj > 0 or wj > 0:
            qj = p[j]
            
            # Neues V
            new_V = list(V)
            if vj > 0:
                new_V[j] -= 1
            
            # Neues W
            new_W = list(W)
            if wj > 0:
                new_W[j] -= 1
            
            total += qj * expected_duel(tuple(new_V), tuple(new_W), p)
            s += qj

    return (1 + total) / s


---

# Eingabe #

---

In [15]:
from fractions import Fraction

# Eingaben
m = int(input("Anzahl der Fächer (z. B. 3): "))

print(f"Gib {m} Wahrscheinlichkeiten als Brüche ein (z. B. 1/2):")
p = [Fraction(input(f"p_{i+1} = ")) for i in range(m)]

# Normalisierung
if sum(p) != 1:
    p = [x / sum(p) for x in p]

print(f"Gib {m} Zahlen für Verteilung V (Spieler A):")
V = tuple(map(int, input("V = ").split()))

print(f"Gib {m} Zahlen für Verteilung W (Spieler B):")
W = tuple(map(int, input("W = ").split()))

# Berechnung
evw = expected_duel(V, W, tuple(p))
print(f"\nErwartete Spieldauer E({V}, {W}) = {evw} ≈ {float(evw):.5f}")


Anzahl der Fächer (z. B. 3):  3


Gib 3 Wahrscheinlichkeiten als Brüche ein (z. B. 1/2):


p_1 =  172
p_2 =  1/3
p_3 =  1/6


Gib 3 Zahlen für Verteilung V (Spieler A):


V =  3 2 1


Gib 3 Zahlen für Verteilung W (Spieler B):


W =  4 2 0



Erwartete Spieldauer E((3, 2, 1), (4, 2, 0)) = 353578057585324135454857529864563237109333/614907909166806869255730876933251865000 ≈ 575.00977
